In [2]:
import numpy as np
import sqlite3
import json
import polyline
import folium
from datetime import datetime


In [3]:
# make heatmap
from folium.plugins import HeatMap

# --- 1. Connect to your SQLite database ---
conn = sqlite3.connect("strava_cache.db")
cursor = conn.cursor()

# --- 2. Fetch all activities with a summary polyline ---
cursor.execute("SELECT data FROM activities")
rows = cursor.fetchall()

heatmap_coords = []

# --- 3. Loop through activities, decode polylines ---
for (data_json,) in rows:
    try:
        data = json.loads(data_json)
        poly = data.get("map", {}).get("summary_polyline")

        if poly:
            coords = polyline.decode(poly)  # returns list of (lat, lon)
            heatmap_coords.extend(coords)  # add all points to list

    except Exception as e:
        print(f"Error parsing activity: {e}")

print(f"Total points: {len(heatmap_coords)}")

# --- 4. Convert coordinates to numpy array for easier calculation ---
coords_array = np.array(heatmap_coords)

# --- 5. Define grid parameters for density calculation ---
grid_size = 0.003  # 0.05 degrees grid size (roughly 5 km grid at equator) 0.003 seems to be the limit (333m x 255m) at 40deg N
lat_min, lon_min = np.min(coords_array, axis=0)
lat_max, lon_max = np.max(coords_array, axis=0)

# --- 6. Create a grid and calculate point density ---
lat_bins = np.arange(lat_min, lat_max, grid_size)
lon_bins = np.arange(lon_min, lon_max, grid_size)

# Create a 2D histogram (density) for the grid
density_grid, _, _ = np.histogram2d(
    coords_array[:, 0], coords_array[:, 1], bins=[lat_bins, lon_bins]
)


# --- 7. Find the grid cell with the highest density ---
max_density_idx = np.unravel_index(np.argmax(density_grid), density_grid.shape)
hottest_lat = lat_bins[max_density_idx[0]] + grid_size / 2  # center of grid cell
hottest_lon = lon_bins[max_density_idx[1]] + grid_size / 2  # center of grid cell

# --- 4. Create Folium map centered on first point ---
if not heatmap_coords:
    raise ValueError("No coordinates found!")
# 
m = folium.Map(location=[hottest_lat, hottest_lon], zoom_start=12, tiles="OpenStreetMap")

# --- 9. Add the heatmap layer ---

HeatMap(heatmap_coords, 
        radius=3, 
        blur=2, 
        min_opacity=0.5,
        max_zoom=18,).add_to(m)


hottest_val = np.max(density_grid)
percent_gpx_hits = hottest_val/len(heatmap_coords) * 100
folium.Marker(
    location=[hottest_lat, hottest_lon],
    popup=f'Hottest  {percent_gpx_hits:.3f} percent',
    icon=folium.Icon(color="red", icon="fire", prefix="fa",
    )
).add_to(m)

# Define grid cell bounds
lat0 = lat_bins[max_density_idx[0]]
lat1 = lat0 + grid_size
lon0 = lon_bins[max_density_idx[1]]
lon1 = lon0 + grid_size

# Add rectangle to map
folium.Rectangle(
    bounds=[(lat0, lon0), (lat1, lon1)],
    color="black",
    weight=2,
    fill=True,
    fill_opacity=0.2,
    popup="Hottest Grid Cell"
).add_to(m)


# --- 6. Save to HTML ---
m.save("heatmap.html")
print("Saved map to heatmap.html")


Total points: 609719
Saved map to heatmap.html


In [3]:
# make sharp map
# --- 1. Connect to the database ---
conn = sqlite3.connect("strava_cache.db")
cursor = conn.cursor()

# --- 2. Fetch all activity data ---
cursor.execute("SELECT id, data FROM activities")
rows = cursor.fetchall()

# --- 3. Create folium map centered on the first route (will update later) ---
m = folium.Map(location=[0, 0], zoom_start=12,)

# --- 4. Loop through activities and draw each PolyLine ---
for activity_id, data_json in rows:
    try:
        data = json.loads(data_json)
        poly = data.get("map", {}).get("summary_polyline")
        if not poly:
            continue

        coords = polyline.decode(poly)  # list of (lat, lon)

        

        folium.PolyLine(
            locations=coords,
            color="#3388ff",       # You can randomize or color by type if you want
            weight=2,
            opacity=0.7,
            popup=data.get("id", f"Activity {activity_id}")
        ).add_to(m)

    except Exception as e:
        print(f"Error processing activity {activity_id}: {e}")

m.location = coords[0]

# --- 5. Save the result ---
m.save("sharp_map.html")
print("Saved map to sharp_map.html")


Saved map to sharp_map.html


In [18]:
# make animated routes
from folium.plugins import TimestampedGeoJson


# --- Connect to DB ---
conn = sqlite3.connect("strava_cache.db")
cursor = conn.cursor()
cursor.execute("SELECT id, data FROM activities")
rows = cursor.fetchall()

# --- Build GeoJSON features ---
features = []

for activity_id, data_json in rows:
    try:
        data = json.loads(data_json)
        poly = data.get("map", {}).get("summary_polyline")
        start_date = data.get("start_date")
        name = data.get("name", f"Activity {activity_id}")
        activity_type = data.get("type", "Unknown")

        if not (poly and start_date):
            continue

        coords = polyline.decode(poly)
        if len(coords) < 2:
            continue

        # Format time (ISO 8601 with Zulu time)
        timestamp = datetime.fromisoformat(start_date.replace("Z", "+00:00")).isoformat()

        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [[lon, lat] for lat, lon in coords],  # GeoJSON uses [lon, lat]
            },
            "properties": {
                "times": [timestamp for _ in coords],  # Needed for animation
                "style": {
                    "color": "#ff5500" if activity_type == "Run" else "#3388ff",
                    "weight": 3,
                    "opacity": 0.8,
                },
            },
        }

        features.append(feature)

    except Exception as e:
        print(f"Error parsing activity {activity_id}: {e}")

# --- Create folium map centered on first coord ---
if not features:
    raise ValueError("No valid activities found.")

first_coord = features[0]['geometry']['coordinates'][0][::-1]  # lat, lon
m = folium.Map(location=first_coord, zoom_start=12, tiles="OpenStreetMap")

# --- Add animated layer ---
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="P1M",                # 1 day per step
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=10,
    loop_button=True,
    date_options="YYYY-MM-DD",
    time_slider_drag_update=True,
).add_to(m)

# --- Save map ---
m.save("animated_routes.html")
print("Saved animation to animated_routes.html")


Saved animation to animated_routes.html


In [12]:
# make small map
small_ids = [
    12292106042,
    8287258968,
    4380620390,
    14226775715,
    9715432255,
    7496473660,
]
# small_ids = [3196106480]
# small_ids = [
#     12845099290,
#     5072881666,
# 5201206275,
# 4031465477,
# 6192940553,
# 4050372631,
# 5347560472,
# 4267227160,
# 4031467551,
# 4912038964,
# 4231162941,
# 6367957566,
# 5247994439,
# 4034113610,
# 4237197386,
# 4809310797,
# 5347560014,
# 5201196115,
# 5248138836,
# 4247508053,
# 5201197142,
# 10327181396,
# 5072889946,
# 5367936092,
# 4234170972,
# 3722572382,
# 5201201254,
# 5110060648,
# 10366431344,
# 6395676785,
# 5201211509,
# 5373401205,
# 10366427767,
# 4852337784,
# 4044139126,
# 5254427263,
# 5247998594,
# 4767318146,
# 5072897673,
# 3936479900,
# 5110061217,
# 5072880296,
# 4195670699,
# 4809305786,
# 5347559611,
# 5201200322,
# 4298879178,
# 5201205969,
# 4333349074,
# 5072890070,
# 5072893147,
# 3196106461,
# 4550290141,
# 4912036062,
# 3196106465,
# 3196106469,
# 4158838079,
# 3196106474,
# 3196106475,
# 3196106481,
# 3196106483,
# 3196106484,
# 5201205493,
# 3196106486,
# 3196106485,
# 3196106488,
# 3196106492,
# 3995685629,
# 4858970367,
# 3196106496,
# 3196106497,
# 3196106495,
# 10344919811,
# 3196106499,
# 3196106501,
# 3196106502,
# 3196106503,
# 3196106504,
# 6395676420,
# 10748831498,
# 3196106507,
# 3709388554,
# 3196106509,
# 3196106510,
# 3196106512,
# 3196106513,
# 5110056210,
# 3196106518,
# 3196106522,
# 3196106523,
# 3196106524,
# 5367937310,
# 3196106526,
# 3196106528,
# 5347560224,
# 4292074274,
# 3196106530,
# 3196106532,
# 3196106533,
# 4767314218,
# 5110061868,
# 3196106541,
# 3196106542,
# 3196106543,
# 4050373936,
# 3196106545,
# 4237447479,
# 3196106551,
# 3196106553,
# 3196106554,
# 5110059835,
# 3196106555,
# 3196106557,
# 5201214781,
# 3196106559,
# 4809312570,
# 4262850874,
# 4011882306,
# 3196106562,
# 4556143420,
# 3196106565,
# 5347557702,
# 5110057286,
# 3196106568,
# 3196106569,
# 4819402564,
# 3196106563,
# 3196106572,
# 3196106573,
# 3196106575,
# 4550289231,
# 3196106578,
# 5347556697,
# 3995687264,
# 5201208672,
# 4068616547,
# 3995691880,
# 5201207151,
# 5247989105,
# 4912036737,
# 5072881027,
# 5201207687,
# 4277842311,
# 5247982987,
# 5413086604,
# 5072884112,
# 3995688849,
# 5201195925,
# 4912032662,
# 5367935381,
# 4208361880,
# 4011881980,
# 5072895389,
# 5201212318,
# 5201210275,
# 5347558310,
# 4186986415,
# 5072882614,
# 10366429114,
# 4041091005,
# 5072892877,
# 5072882132,
# 4809309661,
# 6371082725,
# 4176842725,
# 4325153254,
# 5413086183,
# 4158837735,
# 5259343338,
# 5347555818,
# 5201203695,
# 3709388278,
# 4068616188,
# 5247984638,

# ]



conn = sqlite3.connect("strava_cache.db")
cursor = conn.cursor()

# Use parameter substitution for safety and formatting
placeholders = ','.join(['?'] * len(small_ids))
query = f"SELECT id, data FROM activities WHERE id IN ({placeholders})"
cursor.execute(query, small_ids)
rows = cursor.fetchall()

# --- 3. Create folium map centered on the first route (will update later) ---
m = folium.Map(location=[0, 0], zoom_start=12,)

# --- 4. Loop through activities and draw each PolyLine ---
for activity_id, data_json in rows:
    try:
        data = json.loads(data_json)
        poly = data.get("map", {}).get("summary_polyline")
        if not poly:
            continue

        coords = polyline.decode(poly)  # list of (lat, lon)

        

        folium.PolyLine(
            locations=coords,
            color="#3388ff",       # You can randomize or color by type if you want
            weight=2,
            opacity=0.7,
            popup=data.get("id", f"Activity {activity_id}")
        ).add_to(m)

    except Exception as e:
        print(f"Error processing activity {activity_id}: {e}")




m.location = coords[0]

# --- 5. Save the result ---
m.save("small_map.html")
print("Saved map to small_map.html")

Saved map to small_map.html


In [10]:
# make MH_combo_map
def make_sharp_map(cache_file,m,color_code):
    # --- 2. Fetch all activity data ---
    conn = sqlite3.connect(cache_file)
    cursor = conn.cursor()
    cursor.execute("SELECT id, data FROM activities")
    rows = cursor.fetchall()
    # --- 4. Loop through activities and draw each PolyLine ---
    
    for activity_id, data_json in rows:
        try:
            data = json.loads(data_json)
            poly = data.get("map", {}).get("summary_polyline")
            if not poly:
                continue

            coords = polyline.decode(poly)  # list of (lat, lon)
            folium.PolyLine(
                locations=coords,
                color=color_code,       # You can randomize or color by type if you want
                weight=2,
                opacity=0.7,
                popup=data.get("id", f"Activity {activity_id}")
            ).add_to(m)

        except Exception as e:
            print(f"Error processing activity {activity_id}: {e}")

golden_ll = [39.754185,-105.230484]
m = folium.Map(location=golden_ll, zoom_start=12,)
make_sharp_map('MH_cache.db',m,"#ff3333")
make_sharp_map('strava_cache.db',m,"#3344ff")



# --- 5. Save the result ---
m.save("MH_combo_map.html")
print("Saved map to MH_combo_map.html")


Saved map to MH_combo_map.html
